## pySheets:  read andwrite pandas data to and from Excel and google Worksheets 

pySheets uses python to transfer data between spreadsheets (XLsx or Google) and easy-to-manipulate pandas dataframes.

* __getDFB(file_or_gsURL)__  retrieves data from a filepath or the URL of a googleSheet.
* __writeDFB(DFB, file_or_gsURL)__ (writes data to a filepath or the URL of a googleSheet.

Thus, 
* To read or write an Excel file, you need its path (e.g., 'myExcelFile.xlsx')
* To read or write a Google Spreadsheet, you need a credentials file, and the URL of the sheet.
    e.g., gs = getGoogleSheet(credentialsFile, URL)
    
    [Here](https://gspread.readthedocs.io/en/latest/oauth2.html) is how to get a credentials file.


We call the resulting data structure a "DataFrameBook")
* An Excel or Google workbook may comprise multiple worksheets, each with it's own name.
* Similarly a DataFrameBook (a DFB) comprises one or more dataframes, as returned by *pd.read_excel*.

    OrderedDict('nameOfFirstDF': firstDF,
                'nameofSecondDF: secondDF')

Then, you can read/write a file or spreadsheet to or from a DFB using __the only two functions you should have to use__ (assuming you have your credentials file, as per the first cell.)
* __getDFB(file_or_gsURL)__
* __writeDFB(DFB, file_or_gsURL)__

<img src='images/diagram.jpg' width=70% height=70%>

In [1]:
#!pip3 install -U -r requirements.txt
from IPython.display import display, HTML
TESTING=True
CREDENTIALSFILE = 'enablebadger-b31383b767ef.json'

### utilities

In [2]:
#!pip3 install arrow
def log(*args):
    msg = [str(arg) for arg in args]
    msg=' '.join(msg)
    import arrow
    log=open('pySheets.log.txt', 'a')
    output=str(arrow.now()).split('.')[0] + ' pySheets.py: '   + msg
    log.write( output + '\n' )
    print( output )

if TESTING: log('abc','def', 2)

2019-04-02T01:16:14 pySheets.py: abc def 2


#### showDF (and illustrate relation between dataframe and array of dicts)

In [3]:
import pandas as pd
def showDF(df):
    display(HTML(df.to_html()))
    
if TESTING:
    arrayOfDicts = [dict(a=1,b=2),
                dict(a=3,b=4),
                dict(a=5,b=6)]
    
    df = pd.DataFrame(arrayOfDicts)
    showDF(df)
    print('showDF renders dataframes even they are not the last item in a cell' )

,a,b
0,1,2
1,3,4
2,5,6


showDF renders dataframes even they are not the last item in a cell


In [4]:
dfb=dict(Sheet1= df, Copy=df)

def showDFB(dfb,msg='showDFB: '):
    keys=list(dfb.keys())
    display(HTML('<i><b>' + msg + '</i></b>' + '  This DataFrameBook contains these sheets:' + str(keys)))
    for key in keys:
        display(HTML('<hr/>'))
        display(HTML('<i>'+ key + '</i>'))
        showDF(dfb[key])
        
if TESTING:
    showDFB(dfb)

,a,b
0,1,2
1,3,4
2,5,6


,a,b
0,1,2
1,3,4
2,5,6


## DBF <-> XL

In [5]:
#!pip3 install XlsxWriter

def DFB_to_XL(DFB, fileName='test.xlsx'):
    #https://xlsxwriter.readthedocs.io/example_pandas_multiple.html
    writer = pd.ExcelWriter(fileName, engine='xlsxwriter')
    for name, df in DFB.items():
        df.to_excel(writer, sheet_name=name, index=False)
    writer.save()
    log('DFB_to_XL:  Wrote to', fileName)

def XL_to_DFB(fileName='df2gSheetTester.xlsx'):
    """take an xlsx file and return an ordered dict of DataFrames (DFB)"""
    log('xl_to_DFB:  Reading', fileName)
    return pd.read_excel(fileName,sheet_name=None)

def test_DFB_XL_DFB():
    DFB_to_XL(dfb,'test.xls')
    new_dfb = XL_to_DFB('test.xls')
    showDFB(dfb, 'original dfb')
          
    showDFB(new_dfb, 'reconstructed dfb')
    
    print('\nalso here is df2gSheetTester.xlsx')
    showDFB(XL_to_DFB())
 
if TESTING:  
    test_DFB_XL_DFB()
    

2019-04-02T01:16:15 pySheets.py: DFB_to_XL:  Wrote to test.xls
2019-04-02T01:16:15 pySheets.py: xl_to_DFB:  Reading test.xls


,a,b
0,1,2
1,3,4
2,5,6


,a,b
0,1,2
1,3,4
2,5,6


,a,b
0,1,2
1,3,4
2,5,6


,a,b
0,1,2
1,3,4
2,5,6



also here is df2gSheetTester.xlsx
2019-04-02T01:16:15 pySheets.py: xl_to_DFB:  Reading df2gSheetTester.xlsx


,numbers,letters
0,1,a
1,2,b
2,3,c


,col1
0,eeny
1,meeny
2,miney
3,moe


## enter GoogleSheets....

In [10]:
#!pip3 install pygsheets
import pygsheets

def getGoogleSheet(URLofSheet='https://docs.google.com/spreadsheets/d/1y19PMDTSqZkpW3UcwqG81tQMAH_g91iIqjrmBN_jTRw/edit#gid=0',
                  credentialsFile = CREDENTIALSFILE):
    """
    1. Have credentials in credentials fileName.  [get google drive api credentials]
    2. Sheet must exist at URLofSheet
    3. Sheet must be shared with the email addess in the credentialsFile
    """
    googleClient = pygsheets.authorize(service_file=credentialsFile)
    log('getGoogleSheet:  ', googleClient)
    return googleClient.open_by_url(URLofSheet)

def test_getGoogleSheet():
    gs = getGoogleSheet()
    print(gs.worksheets())

if TESTING: 
    test_getGoogleSheet()

2019-04-02T01:18:14 pySheets.py: getGoogleSheet:   <pygsheets.client.Client object at 0x7f3770da9780>
[<Worksheet 'SheetTheFirst' index:0>, <Worksheet 'SheetTheSecond' index:1>]


### DFB <-> GS

In [11]:
def GS_to_DFB(gsURL):
    """take the URL of a Gsheet and return a DFB"""
    gs=getGoogleSheet(gsURL)
    from collections import OrderedDict
    OD = OrderedDict()
    for worksheet in gs.worksheets():
        title = worksheet.title
        df = pd.DataFrame(worksheet.get_all_records())
        OD[title] = df
    return OD

def test_GS_to_DFB():
    URL = 'https://docs.google.com/spreadsheets/d/1y19PMDTSqZkpW3UcwqG81tQMAH_g91iIqjrmBN_jTRw/edit#gid=0'
    dfb = GS_to_DFB(URL)
    showDFB(dfb, URL)

if TESTING: 
    test_GS_to_DFB()

2019-04-02T01:18:18 pySheets.py: getGoogleSheet:   <pygsheets.client.Client object at 0x7f3770e5d6d8>


,letters,numbers
0,a,1
1,b,2
2,c,3


,col1
0,eeny
1,meeny
2,miney
3,moe


In [12]:
def DFB_to_GS(DFB, gsURL):
    gs = getGoogleSheet(gsURL)
    while len(DFB)>len(gs.worksheets()): #add sheets if necessary
        gs.add_worksheet('temp',1,1)
    for i,title in enumerate(DFB.keys()):
        gs[i].clear()
        gs[i].title = title
        gs[i].set_dataframe(DFB[title],(1,1))# insert the data
    log('DFB_to_GS: updated', gs.title)

def test_DFB_to_GS(): #test: sheet in cloud changes
    DFB = XL_to_DFB('test2.xls')
    gsURL='https://docs.google.com/spreadsheets/d/1y19PMDTSqZkpW3UcwqG81tQMAH_g91iIqjrmBN_jTRw/edit#gid=1856401945'
    DFB_to_GS(DFB,gsURL)

if TESTING: 
    test_DFB_to_GS()

2019-04-02T01:18:21 pySheets.py: xl_to_DFB:  Reading test2.xls
2019-04-02T01:18:21 pySheets.py: getGoogleSheet:   <pygsheets.client.Client object at 0x7f3770c4ef28>
2019-04-02T01:18:24 pySheets.py: DFB_to_GS: updated pySheetsTester


# get_DFB 

In [13]:
def get_DFB(fileName_or_gsURL):
    log('get_DFB:  working on', fileName_or_gsURL)
    if fileName_or_gsURL.startswith('http'):
        return GS_to_DFB(fileName_or_gsURL)
    else:
        return XL_to_DFB(fileName_or_gsURL)

def test_get_DFB():
    #dfb =  get_DFB('test2.xls')
    #showDFB(dfb, 'DFB from test2.xls')
    
    dfb = get_DFB('https://docs.google.com/spreadsheets/d/1y19PMDTSqZkpW3UcwqG81tQMAH_g91iIqjrmBN_jTRw/edit#gid=0')
    showDFB(dfb, 'DFB from google Sheet')

if TESTING: 
    test_get_DFB()

2019-04-02T01:18:24 pySheets.py: get_DFB:  working on https://docs.google.com/spreadsheets/d/1y19PMDTSqZkpW3UcwqG81tQMAH_g91iIqjrmBN_jTRw/edit#gid=0
2019-04-02T01:18:24 pySheets.py: getGoogleSheet:   <pygsheets.client.Client object at 0x7f3770e5d320>


,letters,numbers
0,a,1
1,b,2
2,c,3


,col1
0,eeny
1,meeny
2,miney
3,moe


# write_DFB

In [14]:
def write_DFB(dfb, fileName_or_gsURL):
    log('write_DFB processing', dfb.keys(), fileName_or_gsURL)
    if fileName_or_gsURL.startswith('http'):
        DFB_to_GS(dfb,fileName_or_gsURL)
    else:
        DFB_to_XL(dfb,fileName_or_gsURL)


def test_write_DFB():
    dfb = XL_to_DFB('df2gSheetTester.xlsx')
    write_DFB(dfb,'test2.xls')
    
    gsURL='https://docs.google.com/spreadsheets/d/1y19PMDTSqZkpW3UcwqG81tQMAH_g91iIqjrmBN_jTRw/edit#gid=1856401945'
    write_DFB(dfb,gsURL)

if TESTING: 
    test_write_DFB()

2019-04-02T01:18:27 pySheets.py: xl_to_DFB:  Reading df2gSheetTester.xlsx
2019-04-02T01:18:27 pySheets.py: write_DFB processing odict_keys(['SheetTheFirst', 'SheetTheSecond']) test2.xls
2019-04-02T01:18:27 pySheets.py: DFB_to_XL:  Wrote to test2.xls
2019-04-02T01:18:27 pySheets.py: write_DFB processing odict_keys(['SheetTheFirst', 'SheetTheSecond']) https://docs.google.com/spreadsheets/d/1y19PMDTSqZkpW3UcwqG81tQMAH_g91iIqjrmBN_jTRw/edit#gid=1856401945
2019-04-02T01:18:27 pySheets.py: getGoogleSheet:   <pygsheets.client.Client object at 0x7f379eb669e8>
2019-04-02T01:18:29 pySheets.py: DFB_to_GS: updated pySheetsTester
